# Action Orchestration

In this example, we will demonstrate how to use ActionWeaver to orchestrate the design of hierarchies and chains of action.

**Keep in mind that orchestration is limited to a single action invoked per API call. If multiple actions are invoked, same set of actions will be available to LLM again**

In [1]:
import os
from typing import List

from agentkit import action
from agentkit import chain_completion

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

**Define function you want model to invoke**

In [2]:
@action(name="FileHandler")
def handle_file(instruction: str) -> str:
    """
    Handles user instructions related to file operations. Put every context in the instruction only!

    Args:
        instruction (str): The user's instruction about file handling.

    Returns:
        str: The response to the user's question.
    """
    print ("Handling file...")
    return instruction


@action(name="ListFiles")
def list_all_files_in_repo(repo_path: str ='.') -> List:
    """
    Lists all the files in the given repository.

    :param repo_path: Path to the repository. Defaults to the current directory.
    :return: List of file paths.
    """
    print (f"List all files in {repo_path}...")

    file_list = []
    for root, _, files in os.walk(repo_path):
        for file in files:
            file_list.append(os.path.join(root, file))
        break
    return file_list

@action(name="ReadFile")
def read_from_file(file_path: str) -> str:
    """
    Reads the content of a file and returns it as a string.

    :param file_path: The path to the file that needs to be read.
    :return: A string containing the content of the file.
    """
    print (f"Read file from {file_path}")
    with open(file_path, 'r') as file:
        content = file.read()
    return f"The file content: \n{content}"



### Build a hierarchy of actions
Once FileHandler invoked by LLM, LLM can access actions from ListFiles, ReadFile, or reply with a message.
```
FileHandler
  |
  |-- ListFiles
  |
  |-- ReadFile
```


In [8]:
response = chain_completion(
  model="gpt-4o",
  messages=[{"role": "user", "content": "list all files"}],
  actions = [handle_file],
  orch = {
        handle_file.name: [read_from_file, list_all_files_in_repo]
  },
  stream=False,
)

response

Handling file...
List all files in ....


ModelResponse(id='chatcmpl-9v5vQNc6NvR62mV7dJbmswBIqpr3w', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Here is a list of all the files:\n\n1. langsmith.ipynb\n2. extract_tabular_data.ipynb\n3. pydantic.ipynb\n4. parallel_tools.ipynb\n5. stateful_agent.ipynb\n6. function_validation_and_exception_handling.ipynb\n7. anyscale.ipynb\n8. ReAct.ipynb\n9. knowledge_graph_extraction.ipynb\n10. quickstart.ipynb\n11. cookbook.ipynb\n12. logging.ipynb\n13. orchestration.ipynb', role='assistant', tool_calls=None, function_call=None))], created=1723394500, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_3aa7262c27', usage=Usage(prompt_tokens=235, completion_tokens=121, total_tokens=356), service_tier=None)

In [9]:
print (response.choices[0].message.content)

Here is a list of all the files:

1. langsmith.ipynb
2. extract_tabular_data.ipynb
3. pydantic.ipynb
4. parallel_tools.ipynb
5. stateful_agent.ipynb
6. function_validation_and_exception_handling.ipynb
7. anyscale.ipynb
8. ReAct.ipynb
9. knowledge_graph_extraction.ipynb
10. quickstart.ipynb
11. cookbook.ipynb
12. logging.ipynb
13. orchestration.ipynb
